In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=6fb7bda88787c249f469bd30d2dd2b290a2a242924eb96473f0bda8681134420
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import log, col
from google.colab import drive

In [ ]:
sc = SparkContext()
spark = SparkSession.builder.appName("TF-IDF").getOrCreate()

In [ ]:
# Load input file
lines_rdd = sc.textFile("/content/userdata.txt").zipWithIndex().map(lambda x: (x[1], x[0]))

In [ ]:
# map function
def map_func(line):
    if isinstance(line, tuple):
        line_text = line[1]
        tokens = line_text.strip().split(",")
        results = []
        for token in tokens:
            eachtoken = token.split(" ")
            for i in eachtoken:
                if i.strip():
                    results.append((i.lower(), line[0]))
        return results
    else:
        return []


# reduce function
def reduce_func(values):
    line_counts = {}
    for line_number in values:
        line_counts[line_number] = line_counts.get(line_number, 0) + 1
    sorted_counts = sorted(line_counts.items())
    return sorted_counts

In [ ]:
# Apply the map-reduce transformation to create the inverted index
inverted_index_rdd = lines_rdd.flatMap(map_func).groupByKey().mapValues(reduce_func)

In [ ]:
# Count the occurrences of each word
word_counts_rdd = inverted_index_rdd.map(lambda x: (x[0], len(x[1])))

In [ ]:

max_count = word_counts_rdd.map(lambda x: x[1]).max()
max_words_rdd = word_counts_rdd.filter(lambda x: x[1] == max_count).map(lambda x: x[0])

In [ ]:

max_words_list = max_words_rdd.collect()

In [ ]:
# Print the output in the required format
for word in max_words_list:
    print(f"{word}\t{max_count}")

us	49995
